In [1]:
!git clone https://github.com/facebookresearch/ParlAI.git

fatal: destination path 'ParlAI' already exists and is not an empty directory.


In [2]:
!cd ParlAI; python setup.py develop

running develop
running egg_info
writing parlai.egg-info/PKG-INFO
writing dependency_links to parlai.egg-info/dependency_links.txt
writing entry points to parlai.egg-info/entry_points.txt
writing requirements to parlai.egg-info/requires.txt
writing top-level names to parlai.egg-info/top_level.txt
adding license file 'LICENSE' (matched pattern 'LICEN[CS]E*')
writing manifest file 'parlai.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/parlai.egg-link (link to .)
parlai 1.2.0 is already the active version in easy-install.pth
Installing parlai script to /usr/local/bin

Installed /content/ParlAI
Processing dependencies for parlai==1.2.0
Searching for jsonlines==2.0.0
Best match: jsonlines 2.0.0
Processing jsonlines-2.0.0-py3.7.egg
jsonlines 2.0.0 is already the active version in easy-install.pth

Using /usr/local/lib/python3.7/dist-packages/jsonlines-2.0.0-py3.7.egg
Searching for websocket-server==0.4
Best match: websocket-server 0.4
Processing webso

In [3]:
!pip install flask_ngrok
!pip install PyYAML==5.1

In [4]:
import pexpect
class ChatBot:
    def __init__(self):
        self.child = pexpect.spawn('python ParlAI/parlai/scripts/interactive.py -t blended_skill_talk -mf zoo:blender/blender_90M/model', timeout=None)
        self.child.expect('Enter Your Message:')
        self.personality = self.child.before.decode('utf-8', 'ignore').split('[context]')[1]
    def get_response(self): 
        response = (self.child.before.split(b'1m'))[1].split(b'\x1b')
        return response[0].decode('utf-8')
    def send_request(self, req):
        self.child.sendline(req)
        self.child.expect('Enter Your Message:')

In [5]:
c = ChatBot()

In [6]:
c.send_request("my name is warun")

In [7]:
c.get_response()

'hi warun , nice to meet you . what do you like to do in your spare time ?'

In [8]:
!pip install transformers==4.6.0

In [9]:
!pip install twilio

In [10]:
from flask import Flask, request
from transformers import pipeline
import requests
import pickle
from twilio.twiml.messaging_response import MessagingResponse
from flask_ngrok import run_with_ngrok
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
maxlen = 310
msg_history = ""
stress_level = 0
alpha = 0.875
model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/StressDetector')
num_words = 100000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'
questions = 0
sent = [0,0]

In [13]:
import json

psych_ques = None
with open("/content/drive/MyDrive/Colab Notebooks/intents.json", "r") as read_file:
    psych_ques = json.load(read_file)

labels = list(psych_ques.keys())
curr_label = 0
curr_idx = 0
flag = -1
print(labels)

['Depression', 'Hypomania/mania', 'Dysthymic disorder', 'Generalized anxiety disorder', 'Obsessive-compulsive disorder', 'Delusions and hallucinations', 'Panic attacks', 'Agoraphobia', 'Post-traumatic stress disorder', 'Social phobia', 'Borderline personality disorder']


In [ ]:
from flask import Flask, request
from transformers import pipeline
import requests
import pickle
from twilio.twiml.messaging_response import MessagingResponse
from flask_ngrok import run_with_ngrok
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = None
with open('/content/drive/MyDrive/Colab Notebooks/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
from transformers import pipeline
sentiment = pipeline("sentiment-analysis")
def stress_detect(msg):
    test_sequences = tokenizer.texts_to_sequences([msg])
    test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
    ans = model.predict(test_padded)
    return ans[0][0]

def sentiment_detect(msg):
    result = sentiment(msg)[0]
    sign = 1 if result['label'] == "POSITIVE" else -1
    score = sign*result['score']
    return score

app = Flask(__name__)
run_with_ngrok(app)
c = ChatBot()

@app.route('/bot', methods=['POST','GET'])
def bot():
    incoming_msg = request.values.get('Body', '').lower()
    resp = MessagingResponse()
    msg = resp.message()
    global msg_history
    global questions
    global flag
    global curr_label
    global curr_idx
    global labels    
    global sent
    global stress_level
    if incoming_msg == 'clear':
        msg_history = ''
        incoming_msg = 'hi'
    if(len(msg_history) + len(incoming_msg)) <= 310:
        msg_history+= '. ' + incoming_msg
    else:
        msg_history = incoming_msg      
    
    old = stress_detect(msg_history)
    new = stress_detect(incoming_msg)
    stress_level = alpha*new + (1-alpha)*old

    if flag == 0:
        if incoming_msg == "yes":
            flag = 1
        elif incoming_msg == "no":
            flag = -1
        elif incoming_msg == "done":
            flag = -1
            questions = 0
        else:
            flag = -1
            questions = 10
    questions+=1

    if (flag == -1) and questions <= 5:
        if stress_level < 0.8:
            c.send_request(incoming_msg)
            r = c.get_response()
            msg.body(r)
        else:
            r = f"Your stress level is more than the desired level({stress_level*100}).\nYou should contact a professional. Seek help at this phone number: 1234567890"
            msg.body(r)
        return str(resp)
    elif (flag == -1) and questions>5:
        r = "Do you want to start the session?(Type Yes or No) You will be asked many questions and you have to answer each of them. Whenever you feel like you are done and wanna follow normal conversation, just type 'done'."
        msg.body(r)
        flag = 0
        return str(resp)
    elif flag == 1:
        r = psych_ques[labels[curr_label]][curr_idx]
        msg.body(r)
        num_ques = len(psych_ques[labels[curr_label]])
        sent[0]+=sentiment_detect(incoming_msg)
        sent[1]+=1
        curr_idx+=1
        if curr_idx>=num_ques:
            curr_idx = 0
            curr_label+=1
            flag = 2
        return str(resp)
    else:
        sent[0]+=sentiment_detect(incoming_msg)
        sent[1]+=1
        total_sent = sent[0]/sent[1]
        sent = [0,0]
        r = f"You have been diagnosed with {labels[curr_label-1]} with a confidence score of {total_sent*100}%. Your stress levels have been detected to be {stress_level*100}."
        if curr_label >= len(labels):
            curr_label = 0
            curr_idx = 0
            flag = -1
            questions = 0
            r+="\n\nYou are done with the questions! Thank you! If it has been helpful to you, be sure to thank the creator, Warun Panpaliya."
        else:
            flag = 1

        msg.body(r)
        return str(resp)

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d54f99787764.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [20/May/2021 14:14:56] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:15:09] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:15:41] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:15:52] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:16:05] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:16:23] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:16:31] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:16:40] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:16:47] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:16:53] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:17:44] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:18:18] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:18:38] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:40:07] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:40:55] "POST /bot HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 14:42:37] "PO

In [ ]:
)